# Dataset acquisition

This notebook downloads and uncompresses the dataset.

## S3 bucket

First, change the name of the S3 bucket below.

In [ ]:
bucket = 'bucket'

The dataset is over 30 GB. If the download times out, you can resume it by adding the `-c` flag.

In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/All_Amazon_Review.json.gz

In [ ]:
!gunzip All_Amazon_Review.json.gz

In [ ]:
import boto3
import os
import threading
import sys

s3 = boto3.resource('s3')

from boto3.s3.transfer import TransferConfig
config = TransferConfig(multipart_threshold=1024 * 25, 
                        max_concurrency=10,
                        multipart_chunksize=1024 * 25,
                        use_threads=True)

file_path = 'All_Amazon_Review.json'
key = 'data/raw/All_Amazon_Review.json'

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()
                
s3.Object(bucket, key).upload_file(file_path,
                        ExtraArgs=None,
                        Config=config,
                        Callback=ProgressPercentage(file_path)
                        )